In [ ]:
# libraries
!pip install gspread

!pip install askdata

from askdata import *

In [ ]:
#params
params = receive_parameter(params={})
print(params)
# spredsheet key - Add this email to your GSheet askdata-google-analytics@askdata.iam.gserviceaccount.com to grant security
sheet_url = params["sheets_url"]
credentials = params["credentials"]

_LOG_FORMAT = "[%(filename)s:%(lineno)s - %(funcName)20s() ] - %(asctime)s --> %(message)s"
g_logger = logging.getLogger()
logging.basicConfig(format=_LOG_FORMAT)
g_logger.setLevel(logging.INFO)

In [ ]:
#askdata = Askdata( env='dev', token=token)

import gspread
import pandas as pd
gc = gspread.service_account(filename=credentials)

# Open a sheet from a spreadsheet in one go
wks = gc.open_by_url(sheet_url).sheet1

data = wks.get_all_values()
headers = data.pop(0)

df = pd.DataFrame(data, columns=headers)
if("" in df.columns.tolist()):
    df = df.drop(columns='')
i = 0
columns = df.columns.tolist()
for column in columns:
    if(columns.count(column)>1):
        columns.remove(column)
        column+="_2"
        columns.insert(i, column)
    columns.remove(column)
    column=column.strip()
    columns.insert(i, column)
    i+=1
df.columns=columns

In [ ]:
#df.replace(to_replace={"Valore": ",00 €"}, value="", regex=True, inplace=True)
#df.replace(to_replace={"Valore": '\\.'}, value="", regex=True, inplace=True)
#df = df.astype({"Valore":"int64"})
import re
for i in range(wks.col_count):
    try:
        unformatted = wks.col_values(i, value_render_option='UNFORMATTED_VALUE')
        if(len(unformatted)==0):
            continue
        formatted = wks.col_values(i, value_render_option='FORMATTED_VALUE')
        if(formatted == unformatted):
            continue
        else:
            temp_unformatted = unformatted[1:]
            temp_formatted = formatted[1:]
            first_type = []
            other_types = []
            for record in temp_unformatted:
                if(len(first_type)==0):
                    first_type.append(type(record))
                if(type(record)==first_type[0]):
                    first_type.append(type(record))
                else:
                    other_types.append(type(record))
            if(len(other_types)!=0):
                df[formatted[0]] = temp_formatted
            else:
                is_currency = False
                is_date = False
                for record in temp_formatted:
                    is_currency = re.search("[.,$€]", record)
                if(is_currency):
                    df[formatted[0]] = temp_unformatted
                else:
                    new_cols = []
                    for record in temp_formatted:
                        new_cols.append(pd.to_datetime(record))
                    df[formatted[0]] = temp_formatted            
    except Exception as e:
        print(e)
        pass

columnsSteps=[]
columnsSteps.append(df.columns.tolist())